## Extraction

In [11]:
# import necessary libraries
import pandas as pd 
from datetime import datetime, timedelta
import sqlite3

In [12]:
# Read the Online Retail Excel file into a pandas DataFrame
file_path = r'C:\Users\Makena\OneDrive\Desktop\USIU\2nd Summer sem\DWM\End sem\DSA-2040_Practical_Exam_Mitchel_413\Data Warehousing\ETL_Process\raw\Online Retail.xlsx'
df = pd.read_excel(file_path)

# Handle missing values by dropping rows with any missing data
df_clean = df.dropna()

# Convert 'InvoiceDate' column to datetime format
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Display the first few rows to verify
df_clean.head()

C:\Users\Makena\AppData\Local\Temp\ipykernel_11280\1915802556.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Transform

In [13]:
# Calculate TotalSales column
df_clean['TotalSales'] = df_clean['Quantity'] * df_clean['UnitPrice']

print('Rows after dropna:', len(df_clean))

# Remove outliers: Quantity < 0 or UnitPrice <= 0
df_clean = df_clean[(df_clean['Quantity'] >= 0) & (df_clean['UnitPrice'] > 0)]
print('Rows after removing outliers:', len(df_clean))

# Check min and max InvoiceDate
print('InvoiceDate range:', df_clean['InvoiceDate'].min(), 'to', df_clean['InvoiceDate'].max())

# Try to filter data for sales in the last year (from 2024-08-12 to 2025-08-12)
current_date = pd.Timestamp('2025-08-12')
one_year_ago = current_date - pd.Timedelta(days=365)
df_last_year = df_clean[(df_clean['InvoiceDate'] >= one_year_ago) & (df_clean['InvoiceDate'] <= current_date)]
print('Rows after date filter:', len(df_last_year))

# If no rows, use the most recent year in the data
if df_last_year.empty:
    print('No rows in the last year, using most recent year in data instead.')
    max_date = df_clean['InvoiceDate'].max()
    min_date = max_date - pd.Timedelta(days=365)
    df_last_year = df_clean[(df_clean['InvoiceDate'] >= min_date) & (df_clean['InvoiceDate'] <= max_date)]
    print('Rows in most recent year:', len(df_last_year))

# Create customer summary: total purchases and country
customer_summary = df_last_year.groupby('CustomerID').agg(
    TotalPurchases=('TotalSales', 'sum'),
    Country=('Country', 'first')
).reset_index()

# Display the first few rows of the customer summary
customer_summary.head()

C:\Users\Makena\AppData\Local\Temp\ipykernel_11280\2995298126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['TotalSales'] = df_clean['Quantity'] * df_clean['UnitPrice']


Rows after dropna: 406829
Rows after removing outliers: 397884
InvoiceDate range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Rows after date filter: 0
No rows in the last year, using most recent year in data instead.
Rows in most recent year: 384529


,CustomerID,TotalPurchases,Country
0,12346.0,77183.60,United Kingdom
1,12347.0,3598.21,Iceland
2,12348.0,1797.24,Finland
3,12349.0,1757.55,Italy
4,12350.0,334.40,Norway


## Load

In [14]:
# Create SQLite database and load data into fact and dimension tables
import os

# Define absolute database file path
db_dir = r'C:\Users\Makena\OneDrive\Desktop\USIU\2nd Summer sem\DWM\End sem\DSA-2040_Practical_Exam_Mitchel_413\Data Warehousing\ETL_Process'
db_path = os.path.join(db_dir, 'retail_dw.db')

# Ensure the directory exists
os.makedirs(db_dir, exist_ok=True)

# Remove existing database for a clean start (optional)
if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)

# Prepare CustomerDim
customer_dim = customer_summary[['CustomerID', 'Country']].drop_duplicates().copy()
customer_dim.to_sql('CustomerDim', conn, index=False, if_exists='replace')

# Prepare TimeDim
time_dim = df_last_year[['InvoiceDate']].drop_duplicates().copy()
time_dim['TimeID'] = time_dim['InvoiceDate'].astype(str)
time_dim['Year'] = time_dim['InvoiceDate'].dt.year
time_dim['Month'] = time_dim['InvoiceDate'].dt.month
time_dim['Day'] = time_dim['InvoiceDate'].dt.day
time_dim.to_sql('TimeDim', conn, index=False, if_exists='replace')

# Prepare SalesFact
sales_fact = df_last_year[['InvoiceNo', 'StockCode', 'CustomerID', 'Quantity', 'UnitPrice', 'TotalSales', 'InvoiceDate']].copy()
sales_fact['TimeID'] = sales_fact['InvoiceDate'].astype(str)
sales_fact.to_sql('SalesFact', conn, index=False, if_exists='replace')

conn.close()
print(f'Data loaded into SQLite database: {db_path}')

Data loaded into SQLite database: C:\Users\Makena\OneDrive\Desktop\USIU\2nd Summer sem\DWM\End sem\DSA-2040_Practical_Exam_Mitchel_413\Data Warehousing\ETL_Process\retail_dw.db


In [15]:
# Export the contents of the SQLite tables to CSV files

# Define database file path
db_path = r'C:\Users\Makena\OneDrive\Desktop\USIU\2nd Summer sem\DWM\End sem\DSA-2040_Practical_Exam_Mitchel_413\Data Warehousing\ETL_Process\retail_dw.db'

conn = sqlite3.connect(db_path)

# Export SalesFact
sales_fact_df = pd.read_sql_query('SELECT * FROM SalesFact', conn)
sales_fact_df.to_csv('SalesFact.csv', index=False)

# Export CustomerDim
customer_dim_df = pd.read_sql_query('SELECT * FROM CustomerDim', conn)
customer_dim_df.to_csv('CustomerDim.csv', index=False)

# Export TimeDim
time_dim_df = pd.read_sql_query('SELECT * FROM TimeDim', conn)
time_dim_df.to_csv('TimeDim.csv', index=False)

conn.close()
print('CSV files generated: SalesFact.csv, CustomerDim.csv, TimeDim.csv')

CSV files generated: SalesFact.csv, CustomerDim.csv, TimeDim.csv


## Full ETL process

In [16]:
def full_etl_process(excel_path, db_path):
    import pandas as pd
    import sqlite3
    import os

    print('Starting ETL process...')

    # Extraction
    df = pd.read_excel(excel_path)
    print(f'Rows after extraction: {len(df)}')

    # Drop missing values
    df_clean = df.dropna()
    print(f'Rows after dropna: {len(df_clean)}')

    # Convert InvoiceDate to datetime
    df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

    # Transformation
    df_clean['TotalSales'] = df_clean['Quantity'] * df_clean['UnitPrice']
    df_clean = df_clean[(df_clean['Quantity'] >= 0) & (df_clean['UnitPrice'] > 0)]
    print(f'Rows after removing outliers: {len(df_clean)}')
